# 03 - Visualization
## Section 1.1: Defining Mass Extinction

---

**Notebook Purpose**: Generate publication-quality figures for the article.

**Author**: Dennis 'dnoice' Smaltz  
**AI Acknowledgement**: Claude Opus 4  
**Version**: 0.1 (Template)  
**Date**: 2025-12-12  
**Signature**: ︻デ═—··· 🎯 = Aim Twice, Shoot Once!

---

### Figures to Generate

1. **fig_1.1_01**: Extinction rate comparison (current vs. Big Five)
2. **fig_1.1_02**: Timeline of mass extinctions
3. **fig_1.1_03**: Rate comparison with uncertainty

---

## 1. Environment Setup

In [ ]:
# Standard imports
import json
from pathlib import Path

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Paths
SECTION_PATH = Path('../').resolve()
DERIVED_DATA_PATH = SECTION_PATH / 'data' / 'derived'
FIGURES_PATH = SECTION_PATH / 'figures'

# Ensure figures directory exists
FIGURES_PATH.mkdir(parents=True, exist_ok=True)

# Style configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')  # Colorblind-safe palette

# Figure settings
FIGURE_DPI = 300
FIGURE_FORMAT = ['png', 'svg']  # Save both formats

# Custom colors
COLORS = {
    'current': '#d62728',      # Red - current crisis
    'big_five': '#1f77b4',     # Blue - historical
    'background': '#2ca02c',   # Green - background
    'uncertainty': '#7f7f7f',  # Gray - uncertainty
}

print(f"Figures will be saved to: {FIGURES_PATH}")

## 2. Load Data

In [ ]:
# Load derived data from 02_analysis_core.ipynb

# Rate calculations
rate_results = pd.read_csv(DERIVED_DATA_PATH / 'extinction_rate_calculations.csv', index_col=0)

# Comparison data
comparison_df = pd.read_csv(DERIVED_DATA_PATH / 'rate_comparison_big_five.csv')

# Key findings
with open(DERIVED_DATA_PATH / 'key_findings.json', 'r') as f:
    key_findings = json.load(f)

print("Data loaded successfully.")
print(f"\nComparison data shape: {comparison_df.shape}")

## 3. Helper Functions

In [ ]:
def save_figure(fig, name: str, dpi: int = FIGURE_DPI):
    """
    Save figure in multiple formats.
    
    Parameters
    ----------
    fig : matplotlib.figure.Figure
        Figure to save
    name : str
        Base filename (without extension)
    dpi : int
        Resolution for raster formats
    """
    for fmt in FIGURE_FORMAT:
        filepath = FIGURES_PATH / f"{name}.{fmt}"
        fig.savefig(filepath, dpi=dpi, bbox_inches='tight', 
                    facecolor='white', edgecolor='none')
        print(f"Saved: {filepath}")


def add_source_annotation(ax, source: str, fontsize: int = 8):
    """
    Add data source annotation to figure.
    """
    ax.annotate(
        f"Data: {source}",
        xy=(1, 0), xycoords='axes fraction',
        xytext=(-5, 5), textcoords='offset points',
        ha='right', va='bottom',
        fontsize=fontsize, fontstyle='italic',
        color='gray'
    )

## 4. Figure 1.1.01: Extinction Rate Comparison

In [ ]:
# Prepare data for bar chart

# Filter for the events we want to compare
events_to_plot = comparison_df[
    (comparison_df['type'].isin(['Big Five', 'Current', 'Background']))
].copy()

# Use only one current estimate for clarity
events_to_plot = events_to_plot[
    ~events_to_plot['event'].isin(['all_since_1500', 'mammals_since_1900'])
]

# Rename for display
display_names = {
    'vertebrates_since_1500': 'Current\n(Vertebrates)',
    'End-Ordovician': 'End-Ordovician\n(443 Ma)',
    'Late Devonian': 'Late Devonian\n(372 Ma)',
    'End-Permian': 'End-Permian\n(252 Ma)',
    'End-Triassic': 'End-Triassic\n(201 Ma)',
    'End-Cretaceous': 'End-Cretaceous\n(66 Ma)',
    'Background Rate': 'Background\nRate'
}
events_to_plot['display_name'] = events_to_plot['event'].map(display_names)

# Sort by type and rate
type_order = {'Background': 0, 'Big Five': 1, 'Current': 2}
events_to_plot['type_order'] = events_to_plot['type'].map(type_order)
events_to_plot = events_to_plot.sort_values(['type_order', 'rate_emsy'])

print(events_to_plot[['display_name', 'type', 'rate_emsy']])

In [ ]:
# Create figure
fig, ax = plt.subplots(figsize=(12, 6))

# Assign colors by type
colors = events_to_plot['type'].map({
    'Background': COLORS['background'],
    'Big Five': COLORS['big_five'],
    'Current': COLORS['current']
})

# Create bar chart
bars = ax.bar(
    range(len(events_to_plot)),
    events_to_plot['rate_emsy'],
    color=colors,
    edgecolor='black',
    linewidth=0.5
)

# Add error bars for current and background (if available)
for i, (_, row) in enumerate(events_to_plot.iterrows()):
    if pd.notna(row.get('rate_ci_low')) and pd.notna(row.get('rate_ci_high')):
        ax.errorbar(
            i, row['rate_emsy'],
            yerr=[[row['rate_emsy'] - row['rate_ci_low']], 
                  [row['rate_ci_high'] - row['rate_emsy']]],
            color='black', capsize=5, capthick=1.5, linewidth=1.5
        )

# Customize
ax.set_xticks(range(len(events_to_plot)))
ax.set_xticklabels(events_to_plot['display_name'], fontsize=10)
ax.set_ylabel('Extinction Rate (E/MSY)', fontsize=12)
ax.set_title('Current Extinction Rate Compared to Mass Extinction Events', 
             fontsize=14, fontweight='bold')

# Log scale for better visualization
ax.set_yscale('log')
ax.set_ylim(0.1, 1e6)

# Add legend
legend_elements = [
    mpatches.Patch(facecolor=COLORS['background'], edgecolor='black', label='Background'),
    mpatches.Patch(facecolor=COLORS['big_five'], edgecolor='black', label='Big Five'),
    mpatches.Patch(facecolor=COLORS['current'], edgecolor='black', label='Current')
]
ax.legend(handles=legend_elements, loc='upper left')

# Add reference line for background rate
ax.axhline(y=1.0, color=COLORS['background'], linestyle='--', alpha=0.5, 
           label='Background (~1 E/MSY)')

# Add source annotation
add_source_annotation(ax, 'IUCN Red List v2025-2, Barnosky et al. 2011')

plt.tight_layout()
save_figure(fig, 'fig_1.1_01_extinction_rate_comparison')
plt.show()

## 5. Figure 1.1.02: Mass Extinction Timeline

In [ ]:
# Prepare timeline data
timeline_data = comparison_df[comparison_df['type'] == 'Big Five'].copy()
timeline_data = timeline_data.sort_values('age_ma', ascending=False)

# Add current event
current_event = pd.DataFrame([{
    'event': 'Sixth Mass Extinction\n(Ongoing)',
    'age_ma': 0,
    'species_loss_pct': 27.8,  # Current threatened %
    'type': 'Current'
}])
timeline_data = pd.concat([timeline_data, current_event], ignore_index=True)

In [ ]:
# Create timeline figure
fig, ax = plt.subplots(figsize=(14, 6))

# Plot events as bubbles sized by species loss
for _, row in timeline_data.iterrows():
    color = COLORS['current'] if row['type'] == 'Current' else COLORS['big_five']
    size = row.get('species_loss_pct', 30) * 10  # Scale for visibility
    
    ax.scatter(
        row['age_ma'], 0,
        s=size, c=color, edgecolors='black', linewidth=1,
        alpha=0.7, zorder=5
    )
    
    # Add label
    y_offset = 0.15 if _ % 2 == 0 else -0.2
    ax.annotate(
        row['event'],
        xy=(row['age_ma'], 0),
        xytext=(row['age_ma'], y_offset),
        ha='center', va='center' if y_offset > 0 else 'top',
        fontsize=9, fontweight='bold'
    )
    
    # Add percentage
    if pd.notna(row.get('species_loss_pct')):
        pct_offset = y_offset + (0.08 if y_offset > 0 else -0.08)
        ax.annotate(
            f"{row['species_loss_pct']:.0f}% loss",
            xy=(row['age_ma'], pct_offset),
            ha='center', fontsize=8, color='gray'
        )

# Timeline axis
ax.axhline(y=0, color='black', linewidth=2)
ax.set_xlim(500, -20)
ax.set_ylim(-0.4, 0.4)

# Labels
ax.set_xlabel('Millions of Years Ago', fontsize=12)
ax.set_title('Timeline of Mass Extinction Events', fontsize=14, fontweight='bold')

# Remove y-axis (it's just a timeline)
ax.yaxis.set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Add present marker
ax.axvline(x=0, color='red', linestyle='--', alpha=0.5)
ax.annotate('Present', xy=(0, -0.35), ha='center', fontsize=10, color='red')

# Legend for bubble size
ax.annotate('Bubble size = species loss %', xy=(0.02, 0.95), 
            xycoords='axes fraction', fontsize=9, fontstyle='italic')

add_source_annotation(ax, 'IUCN, Barnosky et al. 2011, Raup & Sepkoski 1982')

plt.tight_layout()
save_figure(fig, 'fig_1.1_02_mass_extinction_timeline')
plt.show()

## 6. Figure 1.1.03: Rate Comparison with Uncertainty

In [ ]:
# Create comparison showing uncertainty ranges
fig, ax = plt.subplots(figsize=(10, 6))

# Data
categories = ['Background\nRate', 'Current\n(Conservative)', 'Current\n(Central)', 'Current\n(Liberal)']
rates = [1.0, 10, 100, 1000]  # Simplified for visualization
errors_low = [0.9, 5, 50, 800]
errors_high = [1.0, 15, 150, 1200]

x_pos = range(len(categories))
colors_plot = [COLORS['background'], COLORS['uncertainty'], COLORS['current'], COLORS['uncertainty']]

# Bar plot
bars = ax.bar(x_pos, rates, color=colors_plot, edgecolor='black', linewidth=1)

# Error bars
ax.errorbar(
    x_pos, rates,
    yerr=[
        [r - el for r, el in zip(rates, errors_low)],
        [eh - r for r, eh in zip(rates, errors_high)]
    ],
    fmt='none', color='black', capsize=5, capthick=2, linewidth=2
)

# Annotations
for i, (rate, cat) in enumerate(zip(rates, categories)):
    label = f"{rate}x" if i > 0 else "1x"
    ax.annotate(
        label, xy=(i, rate), xytext=(0, 5),
        textcoords='offset points', ha='center',
        fontsize=11, fontweight='bold'
    )

# Formatting
ax.set_xticks(x_pos)
ax.set_xticklabels(categories, fontsize=10)
ax.set_ylabel('Relative to Background Rate', fontsize=12)
ax.set_title('Current Extinction Rate: Range of Estimates', fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.set_ylim(0.5, 2000)

# Reference line
ax.axhline(y=1, color=COLORS['background'], linestyle='--', linewidth=2, 
           label='Background rate', alpha=0.7)

# Add note
ax.annotate(
    'Conservative = documented extinctions only\n'
    'Liberal = including likely undocumented extinctions',
    xy=(0.02, 0.95), xycoords='axes fraction',
    fontsize=9, fontstyle='italic', va='top'
)

add_source_annotation(ax, 'Barnosky 2011, Ceballos 2015, De Vos 2015')

plt.tight_layout()
save_figure(fig, 'fig_1.1_03_rate_uncertainty_comparison')
plt.show()

## 7. Figure Manifest

In [ ]:
# Generate figure manifest with captions

figure_manifest = {
    'fig_1.1_01_extinction_rate_comparison': {
        'title': 'Current Extinction Rate Compared to Mass Extinction Events',
        'caption': 'Comparison of current vertebrate extinction rate (red) with the '
                   'five major mass extinction events in Earth\'s history (blue) and '
                   'the background extinction rate (green). Current rate calculated '
                   'from documented extinctions since 1500 CE. Error bars show 95% '
                   'confidence intervals where available. Y-axis on logarithmic scale.',
        'data_sources': ['DS-PA-001 (IUCN Red List)', 'DS-PR-001 (Barnosky 2011)'],
        'notebook': '03_visualization.ipynb'
    },
    'fig_1.1_02_mass_extinction_timeline': {
        'title': 'Timeline of Mass Extinction Events',
        'caption': 'Timeline showing the five major mass extinction events over the '
                   'past 500 million years, with the current (sixth) mass extinction '
                   'at present. Bubble size represents estimated species loss percentage. '
                   'Note the current event is ongoing and loss estimates are for '
                   'currently threatened species.',
        'data_sources': ['DS-PA-001', 'DS-PR-001', 'Raup & Sepkoski 1982'],
        'notebook': '03_visualization.ipynb'
    },
    'fig_1.1_03_rate_uncertainty_comparison': {
        'title': 'Current Extinction Rate: Range of Estimates',
        'caption': 'Range of estimates for current extinction rate relative to '
                   'background rate. Conservative estimates include only well-documented '
                   'extinctions; liberal estimates account for likely undocumented '
                   'extinctions, particularly in poorly-surveyed taxa. Error bars '
                   'represent uncertainty ranges from the literature.',
        'data_sources': ['DS-PR-001', 'DS-PR-002', 'De Vos 2015'],
        'notebook': '03_visualization.ipynb'
    }
}

# Save manifest
with open(FIGURES_PATH / 'FIGURE_MANIFEST.json', 'w') as f:
    json.dump(figure_manifest, f, indent=2)

print("Figure Manifest:")
for fig_name, metadata in figure_manifest.items():
    print(f"\n{fig_name}:")
    print(f"  Title: {metadata['title']}")

## 8. Verify All Figures Generated

In [ ]:
# List all generated figures
print("Generated Figures:")
print("=" * 50)

for filepath in sorted(FIGURES_PATH.glob('fig_*')):
    size_kb = filepath.stat().st_size / 1024
    print(f"  {filepath.name} ({size_kb:.1f} KB)")

print("\nManifest:")
for filepath in FIGURES_PATH.glob('*.json'):
    print(f"  {filepath.name}")

---

## Figures Complete

All figures have been generated and saved in both PNG (300 DPI) and SVG formats.

### Next Steps

1. Review figures for quality and accessibility
2. Update article with figure references
3. Complete `uncertainty_documentation.md` and `methods_original_analysis.md`

---

*︻デ═—··· 🎯 = Aim Twice, Shoot Once!*